In [1]:
from datasets import load_dataset

dataset = load_dataset("squad")

Reusing dataset squad (/home/thushv89/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7)


In [2]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


In [3]:
dataset["train"]["answers"][:5]

[{'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 {'answer_start': [188], 'text': ['a copper statue of Christ']},
 {'answer_start': [279], 'text': ['the Main Building']},
 {'answer_start': [381], 'text': ['a Marian place of prayer and reflection']},
 {'answer_start': [92], 'text': ['a golden statue of the Virgin Mary']}]

In [4]:
def correct_indices_add_end_idx(answers, contexts):
    
    n_correct, n_fix = 0, 0
    new_answers = []
    for answer, context in zip(answers, contexts):

        gold_text = answer['text'][0]
        answer['text'] = gold_text
        start_idx = answer['answer_start'][0]
        answer['answer_start'] = start_idx
        end_idx = start_idx + len(gold_text)        
        
        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
            n_correct += 1
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
            n_fix += 1
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
            n_fix +=1
        
        
    print("\t{}/{} examples had the correct answer indices".format(n_correct, len(answers)))
    print("\t{}/{} examples had the wrong answer indices".format(n_fix, len(answers)))
    return answers, contexts

train_questions = dataset["train"]["question"]
print("Training data corrections")
train_answers, train_contexts = correct_indices_add_end_idx(dataset["train"]["answers"], dataset["train"]["context"])
test_questions = dataset["validation"]["question"]
print("\nValidation data correction")
test_answers, test_contexts = correct_indices_add_end_idx(dataset["validation"]["answers"], dataset["validation"]["context"])

Training data corrections
	87341/87599 examples had the correct answer indices
	258/87599 examples had the wrong answer indices

Validation data correction
	10565/10570 examples had the correct answer indices
	5/10570 examples had the wrong answer indices


In [5]:
print(len(train_questions))

87599


In [6]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [7]:
context = "This is the context"
question = "This is the question"

token_ids = tokenizer(context, question, return_tensors='tf')
print(token_ids)
print(tokenizer.convert_ids_to_tokens(token_ids['input_ids'].numpy()[0]))

{'input_ids': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=
array([[ 101, 2023, 2003, 1996, 6123,  102, 2023, 2003, 1996, 3160,  102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
['[CLS]', 'this', 'is', 'the', 'context', '[SEP]', 'this', 'is', 'the', 'question', '[SEP]']


In [8]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
print(len(train_encodings["input_ids"]))
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)
print(len(test_encodings["input_ids"]))


87599
10570


In [9]:
type(train_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [10]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        print(answers[i]['answer_start'])
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(test_encodings, test_answers)

515
188
279
381
92
248
441
598
126
908
119
145
234
356
675
487
46
126
271
155
496
68
155
647
358
624
1163
92
757
4
466
303
377
360
136
145
188
344
394
109
138
213
488
618
32
362
565
155
918
0
353
406
638
85
3
136
123
222
49
0
963
1049
1099
86
0
68
233
4
80
118
427
753
891
71
196
1446
1588
49
6
136
350
32
368
73
197
331
1237
251
702
90
228
385
862
244
595
8
66
430
117
204
354
251
274
297
571
1193
819
842
11
11
292
321
587
428
522
720
4
575
37
181
262
82
439
82
625
921
1199
141
64
314
403
576
191
6
68
138
488
596
162
202
349
474
730
56
73
157
284
535
120
336
398
613
1755
142
471
596
750
198
289
535
210
0
85
122
221
424
78
60
134
242
275
4
159
179
325
624
388
405
538
654
0
162
212
478
519
4
92
220
287
327
62
149
214
372
454
140
294
494
557
887
3
34
111
290
336
51
359
495
516
453
24
40
128
440
42
169
401
249
367
82
293
889
959
384
239
382
674
1223
1398
50
223
392
420
657
74
142
509
715
441
0
362
457
506
562
166
245
594
480
424
30
59
119
154
850
222
173
149
411
677
267
344
513
410
40
185
27

23
400
680
802
187
802
654
678
389
729
389
654
507
695
120
120
515
1373
2118
2982
0
201
252
201
35
257
234
46
46
4
31
206
4
73
171
108
206
407
319
601
20
180
224
319
61
99
205
298
708
99
131
153
205
272
74
317
348
34
317
341
101
139
323
613
101
267
376
577
233
840
172
292
353
684
1290
122
426
128
165
180
348
460
423
95
124
461
335
736
0
90
421
0
210
326
245
424
26
45
90
138
45
5
143
45
123
185
371
456
45
329
626
703
737
71
4
329
329
415
20
193
308
320
691
215
320
596
778
10
137
375
398
35
145
292
375
423
0
156
379
485
0
98
156
253
474
105
125
214
65
105
125
41
40
138
291
63
171
237
390
384
27
278
27
109
129
200
288
44
88
170
95
48
470
834
813
13
94
327
432
4
94
185
269
432
55
85
113
154
270
85
55
113
154
338
30
71
126
133
224
30
120
133
224
395
122
292
338
72
145
376
603
31
111
153
176
155
72
111
153
176
300
81
300
418
450
300
140
418
450
617
29
103
208
104
20
103
208
268
0
22
265
270
36
118
195
68
146
197
27
72
230
312
387
579
143
60
141
220
14
431
101
240
421
457
1123
24
157
232
294


637
164
54
164
231
456
637
86
248
78
248
126
492
0
437
0
492
437
0
94
46
135
329
0
46
135
94
46
97
251
158
97
158
49
251
38
117
117
26
117
117
141
117
278
406
141
117
278
406
219
346
434
20
111
206
67
346
121
121
156
47
11
178
7
181
135
13
3
13
135
284
0
171
0
21
171
0
0
0
0
153
0
231
322
17
93
168
3
17
93
168
48
54
48
54
41
87
29
87
9
227
176
295
9
9
87
176
227
295
26
90
246
26
186
246
29
92
7
7
29
92
37
254
332
0
0
25
98
254
332
363
262
230
39
107
202
262
355
38
328
270
26
328
104
270
153
124
267
78
78
267
124
0
163
279
346
507
33
114
237
346
507
39
39
78
39
256
0
329
256
321
38
258
62
26
26
44
258
64
341
228
289
404
50
64
228
293
352
11
154
41
41
274
11
41
154
23
23
328
260
23
192
260
353
54
93
54
93
167
190
37
88
161
25
133
88
161
0
300
59
0
135
169
300
322
0
45
160
303
0
160
21
10
21
143
31
239
98
35
239
291
50
7
424
288
378
38
378
144
424
0
114
114
35
114
52
170
184
184
52
170
184
273
190
0
229
0
229
79
49
255
322
491
128
49
474
35
137
154
23
154
182
233
9
185
224
9
9
40
0
0
229


71
0
0
148
228
313
112
133
364
68
105
68
751
466
74
263
422
160
190
179
349
426
474
598
49
196
346
48
209
245
290
209
316
622
727
913
159
360
440
598
69
49
27
432
214
43
86
194
223
101
277
396
38
103
53
258
330
503
16
288
364
625
684
3
199
67
144
98
147
266
128
384
529
322
44
117
361
470
852
349
553
593
42
72
290
53
236
88
296
243
354
529
559
3
217
134
671
40
97
190
3
95
57
148
38
159
333
158
311
190
70
88
16
95
7
267
51
138
497
92
1098
497
5
112
477
81
228
135
290
43
1027
131
2155
942
1155
11
240
383
794
21
592
178
750
804
648
459
944
1025
67
212
639
398
200
62
174
86
186
128
27
254
155
45
298
159
32
134
176
227
312
171
64
202
192
230
346
128
635
635
111
29
256
361
121
194
362
530
750
138
599
371
583
154
178
446
91
472
0
108
530
452
81
163
445
27
962
121
38
212
575
715
195
228
3
474
488
714
736
277
36
184
245
213
43
206
112
549
103
325
496
428
694
48
259
378
402
103
52
564
53
0
805
1094
156
25
182
156
811
60
87
143
132
313
220
181
462
311
31
461
528
655
324
3
178
585
99
716
443
171
14

105
132
391
150
384
187
273
0
136
270
35
109
142
161
193
67
4
370
411
540
261
129
113
198
261
242
90
196
464
415
325
81
229
394
463
764
52
543
602
803
56
185
209
314
653
0
88
0
167
238
31
202
466
66
5
97
228
202
389
59
141
264
289
383
11
48
195
439
540
25
23
484
513
527
148
202
242
288
96
43
352
225
247
407
59
118
198
245
301
3
79
163
200
26
102
144
97
156
190
226
371
76
108
230
21
90
129
140
183
3
106
117
3
72
100
111
220
3
55
67
239
452
108
270
307
722
856
12
74
120
183
267
82
127
178
360
449
48
21
128
88
227
95
194
199
9
41
73
181
277
287
13
71
26
28
157
158
265
219
33
193
263
617
412
43
164
355
522
387
41
53
278
278
426
9
186
244
115
49
191
299
331
33
139
218
277
405
53
167
225
359
428
121
3
181
309
435
480
75
48
146
3
471
501
548
80
113
157
198
3
65
3
79
194
257
144
49
4
0
124
248
606
628
6
65
43
170
233
123
151
24
391
489
98
24
152
196
64
59
150
104
4
79
116
174
240
282
0
139
242
310
473
83
121
197
73
128
224
232
166
79
203
306
337
406
51
369
286
330
88
118
199
366
463
69
215
126

...

691
36
936
691
1075
133
19
254
339
637
0
136
110
365
512
61
128
163
111
741
37
322
307
391
530
107
132
356
489
435
4
336
368
404
541
21
98
185
219
291
93
176
272
380
420
81
67
105
317
521
0
122
175
1109
100
139
390
3
145
373
74
88
292
462
488
537
388
506
217
137
83
129
158
6
181
382
283
657
10
186
413
331
64
199
322
521
630
20
129
751
879
54
128
221
460
61
209
484
561
683
84
448
644
938
1000
121
26
319
392
449
6
450
188
517
1031
263
104
0
328
348
55
39
84
114
126
85
115
158
371
205
66
255
549
596
620
44
100
257
350
428
4
135
251
494
86
135
269
516
627
30
418
531
570
87
194
488
521
1094
238
16
470
6
76
208
456
27
152
514
73
66
74
189
145
213
202
260
288
345
79
93
207
657
196
478
392
506
540
96
224
278
305
362
279
96
224
308
365
51
353
268
325
410
87
131
210
824
945
1214
1172
910
1091
36
218
147
254
419
207
344
802
79
412
163
4
47
47
823
90
84
63
219
472
67
121
153
185
217
192
237
200
437
244
84
330
261
310
561
54
167
319
396
358
54
132
432
388
580
185
443
708
38
290
312
380
491
231
315


200
347
483
79
583
284
95
353
595
121
259
429
200
239
358
0
165
424
538
81
144
287
491
209
577
555
586
831
56
105
299
356
383
151
261
92
170
117
207
346
0
218
387
402
705
3
96
107
295
826
872
877
901
302
607
0
438
291
130
155
231
281
412
59
105
202
463
246
177
319
333
288
345
129
211
346
405
533
137
240
290
543
818
85
133
594
619
773
233
334
10
445
47
34
254
254
708
39
51
445
250
333
638
572
0
99
251
306
330
384
234
535
188
17
254
505
575
711
331
1254
1440
1701
0
375
320
415
486
0
200
401
431
462
97
151
196
283
504
25
117
185
397
457
39
141
230
328
451
3
59
85
783
427
0
101
326
140
478
79
95
205
485
347
65
368
43
720
854
305
116
370
217
493
3
117
516
706
5
52
341
309
611
0
117
238
471
610
96
250
324
379
570
357
87
494
255
662
34
143
235
302
922
22
87
431
695
785
0
135
468
611
699
63
143
382
567
1088
0
168
440
789
1123
33
288
337
550
702
23
164
308
480
706
50
108
284
352
513
123
318
451
647
1527
563
144
239
388
98
779
953
191
268
43
47
203
238
467
806
11
99
228
488
37
220
320
1242
825
1

204
30
187
34
11
148
284
217
0
175
133
46
176
305
440
615
59
59
161
86
187
237
402
67
93
190
400
520
43
131
195
349
411
0
179
212
179
179
67
167
339
22
52
0
277
142
95
284
82
199
388
484
551
347
24
156
183
320
0
427
924
117
0
91
130
398
375
49
330
108
125
248
273
382
110
76
402
468
261
695
873
85
106
170
248
194
194
4
121
91
154
26
155
251
178
399
123
434
676
52
80
281
177
16
90
40
122
218
318
398
597
65
52
15
43
150
209
234
357
113
220
369
472
714
4
186
268
59
122
350
469
511
38
207
109
364
392
315
121
241
503
244
244
513
244
21
167
211
361
472
227
365
595
16
134
164
0
217
381
424
637
0
224
323
641
34
102
269
403
22
115
167
298
30
101
162
206
237
158
673
605
219
276
399
455
539
110
230
211
245
319
137
187
224
265
224
102
215
233
98
104
9
129
433
502
619
328
246
531
66
32
91
101
187
171
59
19
134
204
14
109
133
0
75
155
290
96
155
193
240
150
92
166
218
239
114
307
177
211
21
21
21
123
200
73
23
58
247
555
572
627
74
142
300
229
102
89
758
921
902
1046
96
249
315
48
566
68
165
537
412


28
64
173
459
363
35
109
171
793
552
36
60
104
165
326
72
237
264
470
577
3
59
55
33
463
27
79
161
227
248
49
92
232
243
815
37
166
937
914
1189
132
537
1003
1062
10
93
110
338
311
366
122
258
551
801
51
30
47
75
232
774
3
33
157
698
71
263
431
136
316
47
104
127
233
396
73
145
200
438
224
932
1469
1540
1517
24
183
199
258
294
445
415
328
27
24
149
614
233
207
39
97
146
73
356
51
200
234
265
553
65
217
318
450
511
54
86
242
290
153
0
572
690
134
269
236
75
391
351
202
27
49
222
672
711
61
768
27
449
768
238
355
545
571
381
9
210
368
471
581
124
154
867
1051
959
32
371
592
733
872
693
153
0
960
93
402
717
803
1006
494
130
273
591
430
364
1030
897
517
494
367
64
476
44
621
6
158
452
350
538
95
337
707
53
243
1302
469
80
1357
1138
442
556
157
424
329
429
253
125
93
168
285
572
778
0
212
560
3
355
479
547
123
407
1028
975
212
4
336
4
3
298
644
707
74
319
409
100
78
935
464
1284
462
645
3
164
302
347
393
26
293
322
233
92
327
395
3
397
644
936
63
103
420
496
63
548
641
621
375
524
648
3
181

610
778
8
299
527
706
904
65
157
107
517
607
201
393
470
555
598
170
0
626
448
306
626
19
191
289
563
664
69
225
329
401
611
0
421
149
522
279
15
104
466
159
639
42
188
406
496
754
0
207
350
429
566
0
207
429
569
71
119
214
423
517
104
83
112
252
572
79
157
202
357
435
56
488
183
247
512
4
102
166
279
482
101
13
195
309
594
4
142
82
290
489
9
90
326
530
553
39
222
337
529
601
29
350
152
739
751
6
97
121
235
452
68
156
273
368
486
46
95
571
466
782
108
459
627
1003
694
0
83
239
269
342
29
69
91
273
461
35
160
338
509
598
681
232
182
124
892
95
128
262
220
587
0
188
335
476
615
3
186
429
870
1094
190
278
506
674
348
93
160
362
255
906
47
264
328
369
359
22
100
181
361
446
19
192
336
419
557
85
188
334
519
660
74
240
450
372
71
401
279
517
450
437
161
888
739
167
948
1349
1751
818
48
1222
392
1003
257
0
347
1061
845
1403
74
526
352
737
940
316
349
266
51
121
30
324
389
106
529
247
449
87
562
315
75
605
738
928
663
76
519
405
303
236
669
324
485
559
448
0
687
510
991
765
48
137
299
446
561

In [26]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: train_encodings[key][:20000] for key in ['input_ids', 'attention_mask']},
    {key: train_encodings[key][:20000] for key in ['start_positions', 'end_positions']}
))

train_dataset = train_dataset.shuffle(1000)

valid_dataset = train_dataset.take(10000)
valid_dataset = train_dataset.map(lambda x, y: (x, (y['start_positions'], y['end_positions'])))
valid_dataset = valid_dataset.batch(16)

train_dataset = train_dataset.skip(10000)
train_dataset = train_dataset.map(lambda x, y: (x, (y['start_positions'], y['end_positions'])))
train_dataset = train_dataset.batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {key: test_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: test_encodings[key] for key in ['start_positions', 'end_positions']}
))
test_dataset = test_dataset.map(lambda x, y: (x, (y['start_positions'], y['end_positions'])))
test_dataset = test_dataset.batch(16)


In [22]:
small_x, small_y = [],[]
for x,y in valid_dataset.take(1):
    small_x = (x['input_ids'], x['attention_mask'])
    small_y = y
    
print()

In [35]:
from transformers import TFBertForQuestionAnswering

#config = DistilBertConfig.from_pretrained("distilbert-base-uncased", return_dict=False)
model = TFBertForQuestionAnswering.from_pretrained("bert-base-uncased")#, config=config)

# Keras will assign a separate loss for each output and add them together. So we'll just use the standard CE loss
# instead of using the built-in model.compute_loss, which expects a dict of outputs and averages the two terms.
# Note that this means the loss will be 2x of when using TFTrainer since we're adding instead of averaging them.
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()
#model.distilbert.return_dict = False # if using 🤗 Transformers >3.02, make sure outputs are tuples

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=loss, metrics=[None]) # can also use any keras loss fn
model.fit(train_dataset, epochs=3)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


TypeError: in user code:

    /home/thushv89/anaconda3/envs/manning.tf2/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/thushv89/anaconda3/envs/manning.tf2/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/thushv89/anaconda3/envs/manning.tf2/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/thushv89/anaconda3/envs/manning.tf2/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/thushv89/anaconda3/envs/manning.tf2/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        r

In [33]:
from transformers import DistilBertConfig,TFDistilBertForQuestionAnswering
config = DistilBertConfig.from_pretrained("distilbert-base-uncased")#, return_dict=False)
#print(config)
#model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased", config=config)
#model.distilbert.return_dict = False
print(model.distilbert.config)
print(model.distilbert.return_dict)
#model.distilbert.return_dict = False
#
y = model(small_x)

inp = tf.keras.layers.Input()
tf.keras.models.Model()

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "return_dict": false,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.3.3",
  "vocab_size": 30522
}

False
TFQuestionAnsweringModelOutput(loss=None, start_logits=array([[-0.13234961,  0.13992603,  0.45321158, ..., -0.14213793,
        -0.07353254, -0.04290646],
       [-0.2659102 , -0.05684969,  0.25423315, ...,  0.01697906,
         0.03252614,  0.07651817],
       [-0.21573944,  0.2346316 , -0.03017157, ..., -0.0664077 ,
         0.05725618,  0.06844629],
       ...,
       [-0.0340183 ,  0.2821657 ,  0.37141207, ...,  0.0358705